<a href="https://colab.research.google.com/github/SohaHussain/LangChain/blob/main/Embeddings_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Embeddings is a way of representing data as points in space where the locations are semantically meaningful. The word 'semantic' just means meanings, where the location captures something about the meaning of a piece of text.

Modern Embeddings uae a transformer neural network  to compute a context aware representation of each word then take an average of the context aware representations.

In [ ]:
!pip install google-cloud-aiplatform

In [ ]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=122b1abb70d8e5d4d9617785a3275e87924d8af2e964c3eb90278b32af9cff55
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [ ]:

from utils import authenticate
credentials, PROJECT_ID = authenticate() # Get credentials and project ID

ImportError: cannot import name 'authenticate' from 'utils' (/usr/local/lib/python3.10/dist-packages/utils/__init__.py)

In [ ]:
REGION = 'us-central1'

In [ ]:
import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

NameError: name 'PROJECT_ID' is not defined

In [ ]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [ ]:
embedding = embedding_model.get_embeddings(
    ["life"])

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
emb_1 = embedding_model.get_embeddings(
    ["What is the meaning of life?"]) # 42!

emb_2 = embedding_model.get_embeddings(
    ["How does one spend their time well on Earth?"])

emb_3 = embedding_model.get_embeddings(
    ["Would you like a salad?"])

vec_1 = [emb_1[0].values]
vec_2 = [emb_2[0].values]
vec_3 = [emb_3[0].values]

In [ ]:
from google.cloud import bigquery
import pandas as pd

In [ ]:
def run_bq_query(sql):

    # Create BQ client
    bq_client = bigquery.Client(project = PROJECT_ID,
                                credentials = credentials)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True,
                                         use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql,
                                    job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [ ]:
language_list = ["python", "html", "r", "css"]

In [ ]:
so_df = pd.DataFrame()

for language in language_list:

    print(f"generating {language} dataframe")

    query = f"""
    SELECT
        CONCAT(q.title, q.body) as input_text,
        a.body AS output_text
    FROM
        `bigquery-public-data.stackoverflow.posts_questions` q
    JOIN
        `bigquery-public-data.stackoverflow.posts_answers` a
    ON
        q.accepted_answer_id = a.id
    WHERE
        q.accepted_answer_id IS NOT NULL AND
        REGEXP_CONTAINS(q.tags, "{language}") AND
        a.creation_date >= "2020-01-01"
    LIMIT
        500
    """


    language_df = run_bq_query(query)
    language_df["category"] = language
    so_df = pd.concat([so_df, language_df],
                      ignore_index = True)

In [ ]:
import time
import numpy as np

In [ ]:
def generate_batches(sentences, batch_size = 5):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

In [ ]:
# This helper function calls model.get_embeddings() on the batch of data, and returns a list containing the embeddings for each text in that batch.

def encode_texts_to_embeddings(sentences):
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

In [ ]:
batch_embeddings = encode_texts_to_embeddings(batch)


### text generation

In [ ]:
from vertexai.language_models import TextGenerationModel

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001") # text generation model for chat conversations use chat-bison model

ValueError: Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project

In [ ]:
prompt = "I'm a high school student.Recommend me a programming activity to improve my skills."

In [ ]:
print(generation_model.predict(prompt=prompt).text)

You can control the behavior of the language model's decoding strategy by adjusting the temperature, top-k, and top-n parameters.

in neural networks we have raw ouputs called logits and we pass these logits to a softmax function in order to get a probability distribution over classes. Here different classes can be considered the different tokens to be returned to the user. So when we apply temperature what we are doing is that we are taking our softmax function and dividing each of our logits value by the temperature value. So as we decrease the temperature value we are increasing the likelihood of selecting the most probable token.

Another strategy is top_k, where we sample from the top k most probable tokens. The default value for top_k is 40.
You can set top_k to values between 1 and 40.

The last strategy is top_p where we can dynamically set the number of tokens to sample from whose cummalative probability is greater than or equal to p. The default value for top_p is 0.95.

top k -> top p -> temperature

In [ ]:
top_k = 20
top_p = 0.7

In [ ]:
prompt = "Write an advertisement for jackets \
that involves blue elephants and avocados."

In [ ]:
response = generation_model.predict(
    prompt=prompt,
    temperature=0.9,
    top_k=top_k,
    top_p=top_p,
)

Grounding in LLM:


*   Access information outside of training data
*   Integrate with existing databases and business data
*   Mitigate risk of hallucinations



Measuring similarity using embeddings

1.   Euclidean distance(l2 distance)
2.   Cosine Similarity
3.   Dot Product



###semantic search

In [ ]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

In [ ]:
query = ['How to concat dataframes pandas']

In [ ]:
query_embedding = embedding_model.get_embeddings(query)[0].values

NameError: name 'embedding_model' is not defined

In [ ]:
cos_sim_array = cosine_similarity([query_embedding],
                                  list(so_database.embeddings.values))

In [ ]:
index_doc_cosine = np.argmax(cos_sim_array)

In [ ]:
index_doc_distances = distances_argmin([query_embedding],
                                       list(so_database.embeddings.values))[0]

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
from IPython.display import Markdown, display

t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)

display(Markdown(response.text))

### Scale with approximate nearest neighbor search

When dealing with a large dataset, computing the similarity between the query and each original embedded document in the database might be too expensive. Instead of doing that, you can use approximate nearest neighbor algorithms that find the most similar documents in a more efficient way.

These algorithms usually work by creating an index for your data, and using that index to find the most similar documents for your queries. In this notebook, we will use ScaNN to demonstrate the benefits of efficient vector similarity search. First, you have to create an index for your embedded dataset.

In [ ]:
import scann
from utils import create_index

#Create index using scann
index = create_index(embedded_dataset = question_embeddings,
                     num_leaves = 25,
                     num_leaves_to_search = 10,
                     training_sample_size = 2000)

In [ ]:
query = "how to concat dataframes pandas"

In [ ]:
import time

start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
neighbors, distances = index.search(query_embedding, final_num_neighbors = 1)
end = time.time()

for id, dist in zip(neighbors, distances):
    print(f"[docid:{id}] [{dist}] -- {so_database.input_text[int(id)][:125]}...")

print("Latency (ms):", 1000 * (end - start))

In [ ]:
start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
index_doc = np.argmax(cos_sim_array)
end = time.time()

print(f"[docid:{index_doc}] [{np.max(cos_sim_array)}] -- {so_database.input_text[int(index_doc)][:125]}...")

print("Latency (ms):", 1000 * (end - start))